In [1]:
!pip install -q datasets transformers peft accelerate

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 566.1/566.1 kB 11.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 47.7/47.7 MB 40.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 104.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 69.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 43.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 7.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 29.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 8.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 8.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 80.6 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not 

In [2]:
from datasets import load_dataset, concatenate_datasets, Dataset
import pandas as pd

# Load PANDORA splits
print("Loading PANDORA dataset...")
train_ds = load_dataset("jingjietan/pandora-big5", split="train")
val_ds   = load_dataset("jingjietan/pandora-big5", split="validation")
test_ds  = load_dataset("jingjietan/pandora-big5", split="test")
full_pandora_ds = concatenate_datasets([train_ds, val_ds, test_ds])

# Load Sentiment140 CSV
print("Loading Sentiment140 dataset...")
df = pd.read_csv(
    "/kaggle/input/sentiment140/training.1600000.processed.noemoticon.csv",
    encoding="latin-1",
    header=None,
    names=["target","ids","date","flag","user","text"]
)
sentiment140_ds = Dataset.from_dict({"text": df["text"].tolist()})

print("✓ Datasets loaded.")

Loading PANDORA dataset...


README.md: 0.00B [00:00, ?B/s]

data/train-00000-of-00002.parquet:   0%|          | 0.00/163M [00:00<?, ?B/s]

data/train-00001-of-00002.parquet:   0%|          | 0.00/164M [00:00<?, ?B/s]

data/validation-00000-of-00001.parquet:   0%|          | 0.00/81.8M [00:00<?, ?B/s]

data/test-00000-of-00001.parquet:   0%|          | 0.00/102M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/1924201 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/481051 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/601314 [00:00<?, ? examples/s]

Loading Sentiment140 dataset...
✓ Datasets loaded.


In [3]:
full_social_media_ds = concatenate_datasets([full_pandora_ds, sentiment140_ds])

# Clean up columns, keeping only 'text'
columns_to_remove = [col for col in full_social_media_ds.column_names if col != "text"]
if columns_to_remove:
    full_social_media_ds = full_social_media_ds.remove_columns(columns_to_remove)
    
print(f"✓ Combined & cleaned corpus size: {len(full_social_media_ds):,}")

✓ Combined & cleaned corpus size: 4,606,566


In [4]:
from collections import Counter
from datasets import DatasetDict

# Tag each example by source and length bucket
def tag_source_length(example):
    text = example["text"]
    source = "reddit" if len(text.split()) > 30 else "twitter" 
    length = len(text.split())
    bucket = "short" if length < 30 else ("medium" if length < 80 else "long")
    return {"source": source, "bucket": bucket}

print("Tagging dataset for stratified sampling...")
tagged_ds = full_social_media_ds.map(tag_source_length, num_proc=2)

# Count strata sizes
counts = Counter((ex["source"], ex["bucket"]) for ex in tagged_ds)

# Determine sample sizes proportional to strata
total = sum(counts.values())
target = 700_000
stratum_sizes = {
    stratum: max(1, int(count * target / total))
    for stratum, count in counts.items()
}

# Sample each stratum
print("Sampling 700,000 examples based on strata...")
samples = []
for (src,bkt), size in stratum_sizes.items():
    stratum = tagged_ds.filter(lambda x, s=src, b=bkt: x["source"]==s and x["bucket"]==b, num_proc=1)
    sample_size = min(size, len(stratum))
    if sample_size > 0:
        sel = stratum.shuffle(seed=42).select(range(sample_size))
        samples.append(sel)

# Concatenate and shuffle final subset
curated_ds = concatenate_datasets(samples).shuffle(seed=42)
print(f"✓ Curated subset size: {len(curated_ds):,}")

Tagging dataset for stratified sampling...


Map (num_proc=2):   0%|          | 0/4606566 [00:00<?, ? examples/s]

Sampling 700,000 examples based on strata...


Filter (num_proc=1):   0%|          | 0/4606566 [00:00<?, ? examples/s]

Filter (num_proc=1):   0%|          | 0/4606566 [00:00<?, ? examples/s]

Filter (num_proc=1):   0%|          | 0/4606566 [00:00<?, ? examples/s]

Filter (num_proc=1):   0%|          | 0/4606566 [00:00<?, ? examples/s]

✓ Curated subset size: 699,998


In [5]:
from transformers import T5Tokenizer, T5ForConditionalGeneration

model_name = "google/flan-t5-base"
tokenizer  = T5Tokenizer.from_pretrained(model_name)
model      = T5ForConditionalGeneration.from_pretrained(model_name)

print(f"✓ Loaded {model_name}")

2025-10-29 20:57:42.956626: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1761771463.171222      19 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1761771463.225384      19 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


tokenizer_config.json: 0.00B [00:00, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

special_tokens_map.json: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565


config.json: 0.00B [00:00, ?B/s]

model.safetensors:   0%|          | 0.00/990M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

✓ Loaded google/flan-t5-base


In [6]:
def tokenize_fn(examples):
    """
    Tokenize text for T5 span corruption.
    The DataCollatorForT5MLM will automatically create masked inputs and labels.
    """
    return tokenizer(
        examples["text"], 
        truncation=True, 
        max_length=128,
        padding=False  # Dynamic padding in collator
    )

print("Tokenizing for T5 Span Corruption...")
tokenized = curated_ds.map(
    tokenize_fn, 
    batched=True, 
    batch_size=1000,
    remove_columns=["text", "source", "bucket"],  # Remove all extra columns
    num_proc=2
)
print("✓ Tokenization complete")

Tokenizing for T5 Span Corruption...


Map (num_proc=2):   0%|          | 0/699998 [00:00<?, ? examples/s]

✓ Tokenization complete


In [7]:
from peft import get_peft_model, LoraConfig, TaskType

lora_config = LoraConfig(
    r=8,
    lora_alpha=16,
    target_modules=["q","v"],
    lora_dropout=0.05,
    bias="none",
    task_type=TaskType.SEQ_2_SEQ_LM
)
model = get_peft_model(model, lora_config)
model.print_trainable_parameters()
print("✓ LoRA applied")

trainable params: 884,736 || all params: 248,462,592 || trainable%: 0.3561
✓ LoRA applied


In [8]:
import os
from transformers import TrainingArguments
import torch
import numpy as np
from dataclasses import dataclass
from typing import Any, Dict, List

# Single GPU for stability
os.environ["CUDA_VISIBLE_DEVICES"] = "0"

@dataclass
class DataCollatorForT5MLM:
    """
    Data collator for T5 Span Corruption (Masked Language Modeling).
    
    This implements T5's span corruption strategy:
    - Corrupts ~15% of tokens
    - Groups corruptions into spans (avg length = 3)
    - Replaces spans with sentinel tokens <extra_id_0>, <extra_id_1>, etc.
    """
    tokenizer: Any
    noise_density: float = 0.15
    mean_noise_span_length: float = 3.0
    input_length: int = 128
    target_length: int = 128
    pad_token_id: int = 0
    decoder_start_token_id: int = 0

    def __post_init__(self):
        if self.tokenizer.pad_token_id is not None:
            self.pad_token_id = self.tokenizer.pad_token_id
        # T5 uses pad_token_id as decoder_start_token_id
        self.decoder_start_token_id = self.tokenizer.pad_token_id
        
        # Pre-compute sentinel token IDs for T5
        self.sentinel_ids = []
        for i in range(100):  # T5 has 100 sentinel tokens
            # For T5, sentinel tokens are: <extra_id_0>, <extra_id_1>, etc.
            token_id = self.tokenizer.convert_tokens_to_ids(f"<extra_id_{i}>")
            self.sentinel_ids.append(token_id)

    def __call__(self, examples: List[Dict[str, Any]]) -> Dict[str, torch.Tensor]:
        # Stack input_ids from all examples
        batch = {
            "input_ids": [ex["input_ids"] for ex in examples],
            "attention_mask": [ex["attention_mask"] for ex in examples]
        }
        
        # Convert to tensors and pad
        batch_input_ids = torch.nn.utils.rnn.pad_sequence(
            [torch.tensor(ids) for ids in batch["input_ids"]],
            batch_first=True,
            padding_value=self.pad_token_id
        )
        
        batch_attention_mask = torch.nn.utils.rnn.pad_sequence(
            [torch.tensor(mask) for mask in batch["attention_mask"]],
            batch_first=True,
            padding_value=0
        )
        
        batch_size, seq_length = batch_input_ids.shape
        
        # Prepare inputs and labels for span corruption
        input_ids_batch = []
        labels_batch = []
        
        for i in range(batch_size):
            input_ids = batch_input_ids[i]
            attention_mask = batch_attention_mask[i]
            
            # Get actual (non-padded) length
            actual_length = attention_mask.sum().item()
            
            if actual_length <= 1:
                # Too short, skip corruption
                input_ids_batch.append(input_ids)
                labels_batch.append(input_ids.clone())
                continue
            
            # Apply T5 span corruption
            corrupted_input, labels = self._apply_span_corruption(
                input_ids[:actual_length]
            )
            
            # Pad to max length
            if len(corrupted_input) < seq_length:
                corrupted_input = torch.cat([
                    corrupted_input,
                    torch.full((seq_length - len(corrupted_input),), self.pad_token_id)
                ])
            if len(labels) < seq_length:
                labels = torch.cat([
                    labels,
                    torch.full((seq_length - len(labels),), -100)
                ])
            
            input_ids_batch.append(corrupted_input[:seq_length])
            labels_batch.append(labels[:seq_length])
        
        return {
            "input_ids": torch.stack(input_ids_batch),
            "attention_mask": batch_attention_mask,
            "labels": torch.stack(labels_batch)
        }
    
    def _apply_span_corruption(self, input_ids: torch.Tensor):
        """
        Apply T5's span corruption to a sequence.
        """
        length = len(input_ids)
        
        # Calculate number of tokens to corrupt
        num_noise_tokens = int(np.round(length * self.noise_density))
        num_noise_tokens = min(max(num_noise_tokens, 1), length - 1)
        
        # Calculate number of noise spans
        num_noise_spans = int(np.round(num_noise_tokens / self.mean_noise_span_length))
        num_noise_spans = max(num_noise_spans, 1)
        
        # Create mask for corrupted positions
        mask_indices = np.arange(length)
        np.random.shuffle(mask_indices)
        
        # Select positions to corrupt
        noise_mask = np.zeros(length, dtype=bool)
        noise_mask[mask_indices[:num_noise_tokens]] = True
        
        # Group into spans
        corrupted_input = []
        labels = []
        sentinel_id = 0
        
        i = 0
        while i < length:
            if noise_mask[i]:
                # Start of a noise span
                # Use pre-computed sentinel token ID
                sentinel_token = self.sentinel_ids[sentinel_id] if sentinel_id < len(self.sentinel_ids) else self.sentinel_ids[0]
                corrupted_input.append(sentinel_token)
                
                # Collect the span
                span_tokens = []
                while i < length and noise_mask[i]:
                    span_tokens.append(input_ids[i].item())
                    i += 1
                
                # Add to labels
                labels.append(sentinel_token)
                labels.extend(span_tokens)
                
                sentinel_id += 1
            else:
                # Non-corrupted token
                corrupted_input.append(input_ids[i].item())
                i += 1
        
        # Add final sentinel to labels
        final_sentinel = self.sentinel_ids[sentinel_id] if sentinel_id < len(self.sentinel_ids) else self.sentinel_ids[0]
        labels.append(final_sentinel)
        
        return torch.tensor(corrupted_input), torch.tensor(labels)


# Create the data collator
print("Creating T5 span corruption data collator...")
data_collator = DataCollatorForT5MLM(
    tokenizer=tokenizer,
    noise_density=0.15,          # 15% of tokens will be corrupted
    mean_noise_span_length=3.0,  # Average span length of 3 tokens
    input_length=128,            # Match max_length from tokenization
    target_length=128,           # Target sequence max length
    pad_token_id=tokenizer.pad_token_id,
    decoder_start_token_id=tokenizer.pad_token_id
)

print(f"✓ Sentinel tokens initialized: {len(data_collator.sentinel_ids)} tokens")
print(f"  Example: '<extra_id_0>' = {data_collator.sentinel_ids[0]}")
print(f"  Example: '<extra_id_1>' = {data_collator.sentinel_ids[1]}")

training_args = TrainingArguments(
    output_dir="/kaggle/working/flan-t5-base-lora-masked-span-700k",
    overwrite_output_dir=True,
    remove_unused_columns=True,
    
    # Training
    num_train_epochs=1,
    per_device_train_batch_size=16,
    gradient_accumulation_steps=2,
    
    # Optimization
    learning_rate=3e-4,
    warmup_ratio=0.03,
    weight_decay=0.01,
    
    # Speed optimizations
    fp16=True,
    optim="adamw_torch_fused",
    dataloader_num_workers=0,
    dataloader_pin_memory=False,
    
    # Progress bar / logging
    logging_steps=50,
    disable_tqdm=False,
    report_to="none",
    
    # Checkpointing
    save_strategy="steps",
    save_steps=1000,
    save_total_limit=2,
)

# Correct step calculation
total_steps = len(tokenized) // (training_args.per_device_train_batch_size * training_args.gradient_accumulation_steps)
print(f"✓ TrainingArgs set with T5 Span Corruption: {total_steps} total update steps")
print(f"✓ Masking Strategy: 15% tokens corrupted, avg span length = 3")
print(f"✓ Custom DataCollatorForT5MLM created successfully")

Creating T5 span corruption data collator...
✓ Sentinel tokens initialized: 100 tokens
  Example: '<extra_id_0>' = 32099
  Example: '<extra_id_1>' = 32098
✓ TrainingArgs set with T5 Span Corruption: 21874 total update steps
✓ Masking Strategy: 15% tokens corrupted, avg span length = 3
✓ Custom DataCollatorForT5MLM created successfully


In [9]:
from transformers import Trainer

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized,
    data_collator=data_collator,  # T5MLM collator handles masking!
)
print(f"✓ Trainer ready with T5 Span Corruption: {len(tokenized):,} examples")

No label_names provided for model class `PeftModelForSeq2SeqLM`. Since `PeftModel` hides base models input arguments, if label_names is not given, label_names can't be set automatically within `Trainer`. Note that empty label_names list will be used instead.


✓ Trainer ready with T5 Span Corruption: 699,998 examples


In [10]:
import time

print("="*60)
print(f"Starting Masked Span Prediction DAPT")
print(f"Model: {model_name} + LoRA on {len(tokenized):,} texts")
print(f"Strategy: T5 Span Corruption (15% tokens, span_len=3)")
print("="*60)

start = time.time()
trainer.train()
hours = (time.time() - start) / 3600

print("="*60)
print(f"✓ Training done in {hours:.2f} hours")
print("="*60)

Starting Masked Span Prediction DAPT
Model: google/flan-t5-base + LoRA on 699,998 texts
Strategy: T5 Span Corruption (15% tokens, span_len=3)


Passing a tuple of `past_key_values` is deprecated and will be removed in Transformers v4.48.0. You should pass an instance of `EncoderDecoderCache` instead, e.g. `past_key_values=EncoderDecoderCache.from_legacy_cache(past_key_values)`.
/usr/local/lib/python3.11/dist-packages/torch/nn/parallel/_functions.py:70: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn(


Step,Training Loss
50,22.760700
100,21.872400
150,18.071100
200,10.290800
250,6.524500
300,5.383900
350,4.719900
400,4.308700
450,3.969800
500,3.890900


/usr/local/lib/python3.11/dist-packages/torch/nn/parallel/_functions.py:70: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/torch/nn/parallel/_functions.py:70: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/torch/nn/parallel/_functions.py:70: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/torch/nn/parallel/_functions.py:70: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/torch/nn/parallel/_functions.py:70: UserWarning: Was asked t

✓ Training done in 6.74 hours


In [11]:
out_dir = "/kaggle/working/flan-t5-base-social-lora-masked-span"
model.save_pretrained(out_dir)
tokenizer.save_pretrained(out_dir)

print(f"✓ LoRA adapter saved to {out_dir}")
print("\n🎯 Model trained with T5 Span Corruption - ready for aspect term extraction!")

✓ LoRA adapter saved to /kaggle/working/flan-t5-base-social-lora-masked-span

🎯 Model trained with T5 Span Corruption - ready for aspect term extraction!
